In [17]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

In [93]:
data = pd.read_csv('/Users/alfahwun/Downloads/bank.csv')

In [19]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [20]:
data.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

In [94]:
data = data.drop(['day','month','pdays'],axis=1)

In [22]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,673,2,-1,0,unknown,yes


In [70]:
[data[i].unique() for i in data.select_dtypes(include='object').columns ]

[array(['admin.', 'technician', 'services', 'management', 'retired',
        'blue-collar', 'unemployed', 'entrepreneur', 'housemaid',
        'unknown', 'self-employed', 'student'], dtype=object),
 array(['married', 'single', 'divorced'], dtype=object),
 array(['secondary', 'tertiary', 'primary', 'unknown'], dtype=object),
 array(['no', 'yes'], dtype=object),
 array(['yes', 'no'], dtype=object),
 array(['no', 'yes'], dtype=object),
 array(['unknown', 'cellular', 'telephone'], dtype=object),
 array(['unknown', 'other', 'failure', 'success'], dtype=object),
 array(['yes', 'no'], dtype=object)]

In [113]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
def feature_encoder(*labels):
  for label in labels:
    data[label] = encoder.fit_transform(data[label])
    print(f"{label} is encoded.")

In [118]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

def encode_categorical_columns(df, categorical_cols):
    # Initialize OneHotEncoder object
    encoder = OneHotEncoder()
    # Fit the encoder on the specified categorical columns
    encoder.fit(df[categorical_cols])
    # Transform the specified categorical columns using the encoder
    transformed_data = encoder.transform(df[categorical_cols]).toarray()
    # Create a list of column names for the new encoded columns
    encoded_cols = []
    for i, category in enumerate(encoder.categories_):
        encoded_cols.extend(categorical_cols[i] + '_' + value for value in category)
    # Create a new DataFrame with the encoded columns
    encoded_df = pd.DataFrame(transformed_data, columns=encoded_cols)
    # Concatenate the encoded DataFrame with the original DataFrame
    df = pd.concat([df, encoded_df], axis=1)
    # Drop the original categorical columns
    df.drop(columns=categorical_cols, inplace=True)
    # Return the updated DataFrame
    return df


In [119]:
data = encode_categorical_columns(data,['job','contact'])

In [120]:
feature_encoder( 'marital', 'education', 'default', 'housing', 'loan',
       'poutcome', 'deposit')

marital is encoded.
education is encoded.
default is encoded.
housing is encoded.
loan is encoded.
poutcome is encoded.
deposit is encoded.


In [121]:
data.head()

,age,marital,education,default,balance,housing,loan,duration,campaign,pdays,...,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,contact_cellular,contact_telephone,contact_unknown
0,59,1,1,0,2343,1,0,1042,1,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,56,1,1,0,45,0,0,1467,1,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,41,1,1,0,1270,1,0,1389,1,-1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,55,1,1,0,2476,1,0,579,1,-1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,54,1,2,0,184,0,0,673,2,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [122]:
X = data.drop('deposit',axis=1)
y = data['deposit']

In [123]:
X.shape

(11162, 27)

In [124]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size = 0.1,random_state=123)
X_model, X_valid, y_model, y_valid = train_test_split(X_train, y_train, test_size = 0.2,random_state=123)

In [125]:
from keras.callbacks import EarlyStopping
# Set callback functions to early stop training 
mycallbacks = [EarlyStopping(monitor='loss', patience=10)]

In [127]:
X.shape

(11162, 27)

In [131]:
model = Sequential([
    Dense(17,input_dim=27,activation='relu'),
    Dense(34,activation='tanh'),
    Dense(1,activation='sigmoid')]
)


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"]) # Configure the model for training
histroy = model.fit(X_model, y_model, validation_data=(X_valid, y_valid) ,batch_size=128,epochs= 100) #training

model.summary()

Epoch 1/100


2023-03-01 10:51:47.746753: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - ETA: 0s - loss: 0.7189 - accuracy: 0.5309

2023-03-01 10:51:49.189444: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 2s 20ms/step - loss: 0.7189 - accuracy: 0.5309 - val_loss: 0.6503 - val_accuracy: 0.6595
Epoch 2/100
63/63 [==============================] - 1s 11ms/step - loss: 0.6247 - accuracy: 0.6663 - val_loss: 0.5926 - val_accuracy: 0.6899
Epoch 3/100
63/63 [==============================] - 1s 11ms/step - loss: 0.5818 - accuracy: 0.7015 - val_loss: 0.5721 - val_accuracy: 0.7003
Epoch 4/100
63/63 [==============================] - 1s 10ms/step - loss: 0.5646 - accuracy: 0.7188 - val_loss: 0.5552 - val_accuracy: 0.7213
Epoch 5/100
63/63 [==============================] - 1s 12ms/step - loss: 0.5451 - accuracy: 0.7287 - val_loss: 0.5408 - val_accuracy: 0.7337
Epoch 6/100
63/63 [==============================] - 1s 10ms/step - loss: 0.5403 - accuracy: 0.7359 - val_loss: 0.5341 - val_accuracy: 0.7367
Epoch 7/100
63/63 [==============================] - 1s 10ms/step - loss: 0.5322 - accuracy: 0.7405 - val_loss: 0.5344 - val_accuracy: 0.7466
Epoch 8/100
63/63 